In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192")

#STATE

In [3]:
from pydantic import BaseModel, Field

class State(BaseModel):
    query: str = Field(description="User question")
    category: str = Field(description="Category the question belongs to")
    sentiment: str = Field(description="Sentiment of the input query")
    response: str = Field(description="Final response")

#Nodes

In [4]:
from langchain_core.prompts import ChatPromptTemplate

class Nodes:
    def __init__(self, llm):
        self.llm = llm

    async def categorization_node(self, state: State)->State:
        """
        Categorize the customer query into Technical, Billing, or General.
        """
        prompt = ChatPromptTemplate.from_template(
            "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
        )

        chain = prompt | self.llm
        category = await chain.ainvoke(state.query)
        return category
    
    async def sentiment_analyser(self, state: State)->State:
        """
        Analyze the sentiment of customer query as Positive, Negative or Neutral
        """
        prompt = ChatPromptTemplate.from_template(
             "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
        )
        chain = prompt | self.llm
        sentiment = await chain.ainvoke(state.query)
        return  sentiment
    
    async def technical_query_handler(self, state: State)->State:
        """
        Provide technical support response to the query
        """
        prompt = ChatPromptTemplate.from_template(
             "Provide a technical support response to the following query: {query}"
        )

        chain = prompt | self.llm
        response = await chain.ainvoke(state.query)
        return response
    
    async def billing_handler(self, state: State)->State:
        """Provide a billing support response to the query."""
        prompt = ChatPromptTemplate.from_template(
            "Provide a billing support response to the {query}"
        )
        chain = prompt | self.llm
        response = await chain.ainvoke(state.query)
        return response
    
    async def general_query_handler(self, state: State)->State:
        """
        Provide support for general cuatomer query
        """
        prompt = ChatPromptTemplate.from_template(
             "Provide a general support response to the following query: {query}"
        )
        chain = prompt | self.llm
        response = await chain.ainvoke(state.query)
        return response
    
    async def query_escalate(self)-> State:
        """
        Escalate the query to a human agent due to negative sentiment.
        """
        return "The query has been escalated tp human agent due to negative sentiment."
    
    def router(state:State):
        """
        Routes the state messages to proper nodes
        """
        if state.sentiment == "Negative":
            return "query_escalate"
        elif state.category == "Technical":
            return "technical_query_handler"
        elif state.category == "general":
            return "general_query_handler"
        else:
            return "billing_handler"

In [5]:
from langgraph.graph import StateGraph, START, END

class WorkFlow:
    def __init__(self):
        self.graph_builder = StateGraph(State)
        self.nodes = Nodes(llm=llm)

    def customer_support_graph(self):
        """
        Provides all the nodes and edges to create a graph
        """
        self.graph_builder.add_node("categorization_node", self.nodes.categorization_node)
        self.graph_builder.add_node("sentiment_analyser", self.nodes.sentiment_analyser)
        self.graph_builder.add_node("technical_support", self.nodes.technical_query_handler)
        self.graph_builder.add_node("general_support", self.nodes.general_query_handler)
        self.graph_builder.add_node("billing_support", self.nodes.billing_handler)
        self.graph_builder.add_node("escalate", self.nodes.query_escalate)

        self.graph_builder.add_edge(START, "categorization_node")
        self.graph_builder.add_edge("categorization_node", "sentiment_analyser")
        self.graph_builder.add_conditional_edges(
                "sentiment_analyser",
                self.nodes.router,
            {
               "query_escalate": "escalate",
               "billing_handler": "billing_support",
               "general_query_handler": "general_support",
               "technical_query_handler": "technical_support"
            }
        )
        self.graph_builder.add_edge("escalate", END)
        self.graph_builder.add_edge("billing_support", END)
        self.graph_builder.add_edge("general_support", END)
        self.graph_builder.add_edge("technical_support", END)

        return self.graph_builder

    
    def setup_graph(self):
        """
        Builds the graph upon calling
        """
        graph = self.graph_builder.compile()
        return graph

In [6]:
async def main(query: str):
    """
    Starts the workflow with query and provides response
    """
    graph = WorkFlow().setup_graph()
    response = await graph.ainvoke(query)
    return response

In [7]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    import asyncio
    query = input("Enter your query: ")
    result = asyncio.run(main(query))
    print(f"Query: {query}")
    print(f"Category: {result['category']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Response: {result['response']}")
    print("\n")

ValueError: Graph must have an entrypoint: add at least one edge from START to another node